[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/search/hybrid-search/fast-intro/pubmed-bm25.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/search/hybrid-search/fast-intro/pubmed-bm25.ipynb)

# Hybrid Search in Pinecone

Pinecone now supports search across dense and sparse vectors with the introduction of the new hybrid vector index — a first-of-its-kind solution that lets engineers easily build keyword-aware semantic search into their applications. In this notebook, we will demonstrate how to use the new hybrid vector index.



## Install Dependencies

In [1]:
!pip install -qU \
    pinecone-datasets==0.7.0 \
    pinecone-client==3.1.0 \
    pinecone-text \
    torch \
    datasets \
    transformers \
    sentence-transformers \
    requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 33.5 MB/s 
     |████████████████████████████████| 5.3 MB 48.6 MB/s 
     |████████████████████████████████| 85 kB 5.3 MB/s 
     |████████████████████████████████| 115 kB 69.3 MB/s 
     |████████████████████████████████| 163 kB 71.1 MB/s 
     |████████████████████████████████| 212 kB 60.8 MB/s 
     |████████████████████████████████| 127 kB 73.6 MB/s 
     |████████████████████████████████| 7.6 MB 53.3 MB/s 
     |████████████████████████████████| 1.3 MB 58.0 MB/s 
     |████████████████████████████████| 115 kB 72.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=317cbf69abc201565efd5d868d8cabeffaa55139a826350233ad0a90f4ba8594
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transfo

## Initializing the Index

Now we need a place to store these embeddings and enable a efficient vector search through them all. To do that we use Pinecone, we can get a [free API key](https://app.pinecone.io/) and enter it below where we will initialize our connection to Pinecone and create a new index.

In [ ]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'

# configure client
pc = Pinecone(api_key=api_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [ ]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Create the index:

In [ ]:
index_name = "hybrid-test"

In [ ]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

## Load Dataset

We will use a Question Answering dataset specific to the medical domain. This is to show how the hybrid index helps to rank search results better when domain specifc terms are used. We will load the dataset from Huggingface dataset hub as follows:

In [10]:
from datasets import load_dataset

# load the medical dataset from huggingfac dataset hub
pubmed = load_dataset('pubmed_qa', 'pqa_labeled', split='train')
pubmed

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset pubmed_qa downloaded and prepared to /root/.cache/huggingface/datasets/pubmed_qa/pqa_labeled/1.0.0/dd4c39f031a958c7e782595fa4dd1b1330484e8bbadd4d9212e5046f27e68924. Subsequent calls will reuse this data.


Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 1000
})

Let's look at a set of contexts (which we will be storing).

In [11]:
contexts = []
# loop through the context passages
for record in pubmed['context']:
    # join context passages for each question and append to contexts list
    contexts.append('\n'.join(record['contexts']))
# view some of the contexts
for context in contexts[:2]:
    print(f"{context[:300]}...")

Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cel...
Assessment of visual acuity depends on the optotypes used for measurement. The ability to recognize different optotypes differs even if their critical details appear under the same visual angle. Since optotypes are evaluated on individuals with good visual acuity and without eye disorders, differenc...


## Sparse Vectors

We will use the BERT tokenizer to create sparse vectors, i.e. we will take the token IDs.

In [12]:
from transformers import BertTokenizerFast

# load bert tokenizer from huggingface
tokenizer = BertTokenizerFast.from_pretrained(
    'bert-base-uncased'
)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Let's create a sparse vector for a context passage.

In [13]:
# tokenize the context passage
inputs = tokenizer(
    contexts[0], padding=True, truncation=True,
    max_length=512
)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [14]:
# extract the input ids
input_ids = inputs['input_ids']
input_ids

[101,
 16984,
 3526,
 2331,
 1006,
 7473,
 2094,
 1007,
 2003,
 1996,
 12222,
 2331,
 1997,
 4442,
 2306,
 2019,
 15923,
 1012,
 1996,
 12922,
 3269,
 1006,
 9706,
 17175,
 18150,
 2239,
 11934,
 27806,
 1007,
 7137,
 2566,
 29278,
 10708,
 1999,
 2049,
 3727,
 2083,
 7473,
 2094,
 1012,
 1996,
 3727,
 1997,
 1996,
 3269,
 8676,
 1997,
 1037,
 17779,
 6198,
 1997,
 20134,
 1998,
 18323,
 9607,
 4372,
 20464,
 18606,
 2024,
 29111,
 1012,
 7473,
 2094,
 5158,
 1999,
 1996,
 4442,
 2012,
 1996,
 2415,
 1997,
 2122,
 2024,
 29111,
 1998,
 22901,
 15436,
 2015,
 1010,
 7458,
 3155,
 2274,
 4442,
 2013,
 1996,
 12436,
 28817,
 20051,
 5397,
 1012,
 1996,
 2535,
 1997,
 10210,
 11663,
 15422,
 4360,
 2076,
 7473,
 2094,
 2038,
 2042,
 3858,
 1999,
 4176,
 1025,
 2174,
 1010,
 2009,
 2038,
 2042,
 2625,
 3273,
 2076,
 7473,
 2094,
 1999,
 4264,
 1012,
 1996,
 2206,
 3259,
 3449,
 14194,
 8524,
 4570,
 1996,
 2535,
 1997,
 23079,
 10949,
 2076,
 13908,
 2135,
 12222,
 7473,
 2094,
 1999,
 2426

We need to convert this into a dictionary of key to frequency values.

In [15]:
from collections import Counter

# convert the input_ids list to a dictionary of key to frequency values
sparse_vec = dict(Counter(input_ids))
sparse_vec

{101: 1,
 16984: 1,
 3526: 2,
 2331: 2,
 1006: 10,
 7473: 13,
 2094: 13,
 1007: 10,
 2003: 2,
 1996: 13,
 12222: 2,
 1997: 13,
 4442: 7,
 2306: 2,
 2019: 1,
 15923: 1,
 1012: 14,
 12922: 2,
 3269: 3,
 9706: 1,
 17175: 1,
 18150: 1,
 2239: 1,
 11934: 2,
 27806: 2,
 7137: 1,
 2566: 3,
 29278: 2,
 10708: 2,
 1999: 11,
 2049: 1,
 3727: 4,
 2083: 1,
 8676: 1,
 1037: 8,
 17779: 1,
 6198: 1,
 20134: 1,
 1998: 7,
 18323: 1,
 9607: 1,
 4372: 1,
 20464: 2,
 18606: 1,
 2024: 3,
 29111: 2,
 5158: 1,
 2012: 1,
 2415: 1,
 2122: 2,
 22901: 1,
 15436: 1,
 2015: 1,
 1010: 7,
 7458: 1,
 3155: 1,
 2274: 1,
 2013: 1,
 12436: 1,
 28817: 1,
 20051: 1,
 5397: 1,
 2535: 2,
 10210: 2,
 11663: 1,
 15422: 1,
 4360: 1,
 2076: 4,
 2038: 2,
 2042: 2,
 3858: 1,
 4176: 1,
 1025: 2,
 2174: 1,
 2009: 1,
 2625: 1,
 3273: 1,
 4264: 1,
 2206: 1,
 3259: 1,
 3449: 1,
 14194: 1,
 8524: 1,
 4570: 1,
 23079: 6,
 10949: 3,
 13908: 1,
 2135: 1,
 24269: 2,
 2309: 1,
 9890: 1,
 3332: 2,
 2754: 2,
 7053: 1,
 10066: 1,
 2001: 2,
 40

Let's write a function to do this in batches. Notice that we are removing some keys from the dictionary. These are special tokens from the tokenizer which we do not really need when creating sparse vectors.

In [16]:
def build_dict(input_batch):
  # store a batch of sparse embeddings
    sparse_emb = []
    # iterate through input batch
    for token_ids in input_batch:
        # convert the input_ids list to a dictionary of key to frequency values
        d = dict(Counter(token_ids))
        # remove special tokens and append sparse vectors to sparse_emb list
        sparse_emb.append({key: d[key] for key in d if key not in [101, 102, 103, 0]})
    # return sparse_emb list
    return sparse_emb

Let's write another function to help us generate sparse vectors in batches.

In [17]:
def generate_sparse_vectors(context_batch):
    # create batch of input_ids
    inputs = tokenizer(
            context_batch, padding=True,
            truncation=True,
            max_length=512
    )['input_ids']
    # create sparse dictionaries
    sparse_embeds = build_dict(inputs)
    return sparse_embeds

## Dense Vectors

Alongside sparse vectors we must also add dense vectors (as usual). We do this like so:

In [18]:
from sentence_transformers import SentenceTransformer

# load a sentence transformer model from huggingface
model = SentenceTransformer(
    'multi-qa-MiniLM-L6-cos-v1',
    device='cuda'
)
model

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

The model gives us a `384` dimensional dense vector.

## Upsert Documents

Now we can go ahead and generate sparse and dense vectors for the full dataset and upsert them along with the metadata to the new hybrid index. We can do that easily as follows:

In [19]:
from tqdm.auto import tqdm

batch_size = 32

for i in tqdm(range(0, len(contexts), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(contexts))
    # extract batch
    context_batch = contexts[i:i_end]
    # create unique IDs
    ids = [str(x) for x in range(i, i_end)]
    # add context passages as metadata
    meta = [{'context': context} for context in context_batch]
    # create dense vectors
    dense_embeds = model.encode(context_batch).tolist()
    # create sparse vectors
    sparse_embeds = generate_sparse_vectors(context_batch)

    vectors = []
    # loop through the data and create dictionaries for uploading documents to pinecone index
    for _id, sparse, dense, metadata in zip(ids, sparse_embeds, dense_embeds, meta):
        vectors.append({
            'id': _id,
            'sparse_values': sparse,
            'values': dense,
            'metadata': metadata
        })

    # upload the documents to the new hybrid index
    index.upsert(vectors=vectors)

# show index description after uploading the documents
index.describe_index_stats()

  0%|          | 0/32 [00:00<?, ?it/s]

{'namespaces': {'': {'vectorCount': 1000}},
 'dimension': 384,
 'indexFullness': 0,
 'totalVectorCount': 1000}

## Querying

Now we can query the index, providing the sparse and dense vectors of a question, along with a weight for keyword relevance (“alpha”). `Alpha=1` will provide a purely semantic-based search result and `alpha=0` will provide a purely keyword-based result equivalent to BM25. The default value is `0.5`.

Let's write a helper function to execute queries and after that run some queries.

In [20]:
from pinecone_text.hybrid import hybrid_convex_scale

def hybrid_query(question, top_k, alpha):
    # convert the question into a sparse vector
    sparse_vec = generate_sparse_vectors([question])
    # convert the question into a dense vector
    dense_vec = model.encode([question]).tolist()
    dense_vec, sparse_vector = hybrid_convex_scale(
        dense_vec, sparse_vec, alpha=alpha
    )
    # query pinecone with the query parameters
    result = index.query(
        vector=dense_vec,
        sparse_vector=sparse_vec[0],
        top_k=top_k,
        include_metadata=True,
      )
    # return search results as json
    return result

In [21]:
question = "Can clinicians use the PHQ-9 to assess depression in people with vision loss?"

First, we will do a pure semantic search by setting the alpha value as 1.

In [22]:
hybrid_query(question, top_k=3, alpha=1)

{'matches': [{'id': '305',
   'score': 0.732677519,
   'values': [],
   'sparseValues': {},
   'metadata': {'context': 'The gap between evidence-based treatments and routine care has been well established. Findings from the Sequenced Treatments Alternatives to Relieve Depression (STAR*D) emphasized the importance of measurement-based care for the treatment of depression as a key ingredient for achieving response and remission; yet measurement-based care approaches are not commonly used in clinical practice.\nThe Nine-Item Patient Health Questionnaire (PHQ-9) for monitoring depression severity was introduced in 19 diverse psychiatric practices. During the one-year course of the project the helpfulness and feasibility of implementation of PHQ-9 in these psychiatric practices were studied. The project was modeled after the Institute for Healthcare Improvement Breakthrough Series. Two of the 19 practices dropped out during the course of the project.\nBy the conclusion of the study, all rem

The most relevant result from above is the second document with id 711. Now let's try with an alpha value of 0.3.

In [23]:
hybrid_query(question, top_k=3, alpha=0.3)

{'matches': [{'id': '711',
   'score': 0.486920714,
   'values': [],
   'sparseValues': {},
   'metadata': {'context': "To investigate whether the Patient Health Questionnaire-9 (PHQ-9) possesses the essential psychometric characteristics to measure depressive symptoms in people with visual impairment.\nThe PHQ-9 scale was completed by 103 participants with low vision. These data were then assessed for fit to the Rasch model.\nThe participants' mean +/- standard deviation (SD) age was 74.7 +/- 12.2 years. Almost one half of them (n = 46; 44.7%) were considered to have severe vision impairment (presenting visual acuity<6/60 in the better eye). Disordered thresholds were evident initially. Collapsing the two middle categories produced ordered thresholds and fit to the Rasch model (chi = 10.1; degrees of freedom = 9; p = 0.34). The mean (SD) items and persons Fit Residual values were -0.31 (1.12) and -0.25 (0.78), respectively, where optimal fit of data to the Rasch model would have a mea

The most relevant document is now ranked the highest.

# Delete the Index

In [ ]:
pc.delete_index("hybrid-test")

---